In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 22 03:31:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    32W / 250W |  15855MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install --upgrade pip
!pip install transformers datasets
!pip install rouge-score nltk
!pip install huggingface_hub
!pip install nltk
!pip install numpy
#!pip install pattern
!pip install spacy
!pip install textacy
!pip install thinc
#!pip install lemminflect
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━

In [ ]:
!python --version

Python 3.7.13


In [ ]:
import transformers

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
model_checkpoint = "t5-small"

## Load the dataset

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
training_set = load_dataset("xsum", split='train[:20000]')
training_set

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 20000
})

## Use the one under this for backchaintranslation, because google translate API can't take more than 5000 characters

In [ ]:
!pip install googletrans==3.1.0a0
from googletrans import Translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
docs = training_set.filter(lambda x: len(x["document"]) <4900)
docs
print(docs['document'])
print(int(0.1*len(docs)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import googletrans
googletrans.LANGUAGES

{'af': 'afrikaans',
 'am': 'amharic',
 'ar': 'arabic',
 'az': 'azerbaijani',
 'be': 'belarusian',
 'bg': 'bulgarian',
 'bn': 'bengali',
 'bs': 'bosnian',
 'ca': 'catalan',
 'ceb': 'cebuano',
 'co': 'corsican',
 'cs': 'czech',
 'cy': 'welsh',
 'da': 'danish',
 'de': 'german',
 'el': 'greek',
 'en': 'english',
 'eo': 'esperanto',
 'es': 'spanish',
 'et': 'estonian',
 'eu': 'basque',
 'fa': 'persian',
 'fi': 'finnish',
 'fr': 'french',
 'fy': 'frisian',
 'ga': 'irish',
 'gd': 'scots gaelic',
 'gl': 'galician',
 'gu': 'gujarati',
 'ha': 'hausa',
 'haw': 'hawaiian',
 'he': 'hebrew',
 'hi': 'hindi',
 'hmn': 'hmong',
 'hr': 'croatian',
 'ht': 'haitian creole',
 'hu': 'hungarian',
 'hy': 'armenian',
 'id': 'indonesian',
 'ig': 'igbo',
 'is': 'icelandic',
 'it': 'italian',
 'iw': 'hebrew',
 'ja': 'japanese',
 'jw': 'javanese',
 'ka': 'georgian',
 'kk': 'kazakh',
 'km': 'khmer',
 'kn': 'kannada',
 'ko': 'korean',
 'ku': 'kurdish (kurmanji)',
 'ky': 'kyrgyz',
 'la': 'latin',
 'lb': 'luxembourgish

In [ ]:
from chaintranslation import ChainTranslation
import chaintranslation
import numpy as np
chaintranslator = ChainTranslation(docs['document'][:2000])
lst = np.array_split(docs['document'][:2000], 20)
lst_summary = np.array_split(docs['summary'][:2000], 20)
lst_id = np.array_split(docs['id'][:2000], 20)
new_list = []
for i in range(len(lst)):
  lst_translated = chaintranslation.mass_chaintranslation(lst[i].tolist(), "en",['da', 'no'])
  for j in range(len(lst_translated)):
    new_list.append({"document": lst_translated[j], "summary": lst_summary[i][j], "id": lst_id[i][j]})
  print(i)

print(len(new_list))

"""
augmentation_size = int(0.1*len(docs))

for i in range(augmentation_size):
  chaintranslator = ChainTranslation(docs["document"][i])
  try:
    translated = chaintranslator.chain_translate("en")
    training_set = training_set.add_item({"document": translated, 'summary': training_set['summary'][i], 'id': training_set['id'][i]})
    print(i)
  except TypeError:
    continue

training_set
"""


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2000


'\naugmentation_size = int(0.1*len(docs))\n\nfor i in range(augmentation_size):\n  chaintranslator = ChainTranslation(docs["document"][i])\n  try:\n    translated = chaintranslator.chain_translate("en")\n    training_set = training_set.add_item({"document": translated, \'summary\': training_set[\'summary\'][i], \'id\': training_set[\'id\'][i]})\n    print(i)\n  except TypeError:\n    continue\n\ntraining_set\n'

In [ ]:
print(len(training_set))
for ne in new_list:
  training_set = training_set.add_item(ne)

20000


## Use the next one for singularization/pluralization

In [ ]:
from singularplural import SingularPlural

In [ ]:
import numpy as np
singplu = SingularPlural(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = singplu.sing_plur()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][1])
print("BAAAAAAAAAAAAAA: ", new_list[1])

LOOOOOOOOOOOOOL:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.
As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.
One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.
The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.
Police have appealed for information about the attack.
Insp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.
"While the exact cause is still under investigation, it is thought that the fire was started deliberately."
BAAAAAAAAAAAAAA:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Sat

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


## Adjectives

In [ ]:
from periphrasis import PeriPhrasis


In [ ]:
import numpy as np
perip = PeriPhrasis(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = perip.mass_adjectives()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][1])
print("BAAAAAAAAAAAAAA: ", new_list[1])

LOOOOOOOOOOOOOL:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.
As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.
One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.
The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.
Police have appealed for information about the attack.
Insp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.
"While the exact cause is still under investigation, it is thought that the fire was started deliberately."
BAAAAAAAAAAAAAA:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Sat

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


## Question Generation

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 2.20 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/question_generation


In [ ]:
import sentencepiece
import transformers
import spacy
from pipelines import pipeline
import itertools
import numpy as np

nlp = spacy.blank('en')
nlp.add_pipe("sentencizer")
model_qg = pipeline("e2e-qg", model="valhalla/t5-base-e2e-qg")
def run_model(docs):
  new_docs = []
  for doc in nlp.pipe(docs, batch_size=500):
    questions = []
    for sent in doc.sents:
      questions.extend(model_qg(sent))
    new_docs.append(' '.join(itertools.chain(questions)))
  return new_docs

quests = run_model(training_set['document'][:2000])
print(quests[0])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])

#run_model("The fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guest was asked to leave the hotel. As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.")

What is one of the areas worst affected? What is still being assessed? What is ongoing in Hawick? What are many roads in Peeblesshire badly affected by? What is the cause of disruption for trains on the west coast mainline? Where is the Lamington Viaduct located? What river overflowed into Newton Stewart? What caused many businesses and householders to be affected? Who visited the area to inspect the damage? What did Nicola Sturgeon inspect? What was the main shopping thoroughfare on Victoria Street? What did the water breach? Who owns the Cinnamon Cafe? What was the name of the restaurant that was badly affected by the flood? What could have been done to ensure the retaining wall did not fail? How much publicity is there for Dumfries and the Nith? What is it like to be neglected or forgotten? What may not be true but it is perhaps my perspective over the last few days? Why were you not ready to help us a bit more? A flood alert remains in place across the Borders because of what? Peeb

In [ ]:
print(len(quests))

2000


In [ ]:
print(len(training_set))
print(len(quests))
for index, item in enumerate(quests):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


In [ ]:
print(training_set['document'][21900])

What was originally thought to be the value of the stolen items over the 2015 Easter holiday? Who are now seeking the larger sum from the five "ringleaders"? What is the maximum amount of time a person can be sentenced to without parole if they don't pay back the amount they owe? When is the confiscation hearing set to begin? How long is the hearing expected to last? What was Britain's biggest ever burglary? When was the Hatton Garden raid? What did the gang raid after boring into the vault of the now defunct Hatton Garden Safe Deposit Ltd? How much of the valuables remain unrecovered? When did a woman come forward claiming she had lost £7m worth of gold in the raid? John Collins, 76, of Islington; Daniel Jones, 62, and Terry Perkins, 68, of Enfield; and the group's oldest member, Brian Reader, 78, of Dartford, Kent, pleaded guilty to what? Collins, Jones and Perkins were each given a seven-year prison term for what crime? Who was sentenced to six years and three months in prison? Who 

In [ ]:
import transformers
from pipelines import pipeline
import sentencepiece
pipel = pipeline("e2e-qg", model="valhalla/t5-base-e2e-qg")
text = "The fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guest was asked to leave the hotel."
text2 = "As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames."
text3 = "One of the tour groups is from Germany, the other from China and Taiwan."
text4 = " It was their first night in Northern Ireland."
text5 = " The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed."
text6 = "Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned."
text7 = "Polouse has appealed for information about the attack."
text8 = "The fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guest was asked to leave the hotel. As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames. One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland. The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed. Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned. Polouse has appealed for information about the attack."

print(pipel(text2))
print(pipel(text3))
print(pipel(text4))
print(pipel(text5))
print(pipel(text6))
print(pipel(text7))
print(pipel(text8))

['How many buses were engulfed by flames?', 'What were the two buses parked side-by-side in?']
['What country does one of the tour groups come from?']
['What was their first night in Northern Ireland?']
['What did the driver of one of the buses say many of the passengers had left on board?']
['What have both groups organised?', 'What will the two groups begin their tour of later than they had planned?']
['What has Polouse appealed for?']
['When did the fire alarm go off at the Holiday Inn in Hope Street?', 'Who was asked to leave the hotel?', 'What was the name of the bus that was engulfed by flames?']


## Synonyms

In [ ]:
import random
import spacy
import textacy
import numpy as np
from synonym import Synonym
import synonym
from textacy.augmentation.utils import AugTok

ModuleNotFoundError: ignored

In [ ]:
syno = Synonym(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = syno.mass_synonyms()

In [ ]:
rs = textacy.resources.ConceptNet()
rs.download()

In [ ]:
text = textacy.spacier.core.make_spacy_doc(
    "The fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guest was asked to leave the hotel. As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames. One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland. The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed. Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned. Polouse has appealed for information about the attack.",
    "en_core_web_sm")
syno2 = synonym.synonym_words(text, ["NOUN", "VERB", "ADV", "ADJ"], 0.75)
synonym.make_new_text(syno2)

33982477it [04:21, 130129.28it/s]


'The firing warning device went off at the Holiday Inn in Hope Street at not far from 04:20 BST on Saturday and edgar albert guest was asked to go away the hotel. As they gathered outside they saw the two buses, parked side of meat-by-side of meat in the car mungo park, engulfed by flames. One of the tour groups is from Germany, the foreign from China and Taiwan. It was their world class dark in Northern Ireland. The driver of one of the buses said many another of the passengers had left personly personal effects on circuit board and these had been destroyed. Both groups have organised replacement coaches and will lead off their go of the north slide by and by than they had planned. Polouse has appealed for selective information about the tone beginning.'

In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][1])
print("BAAAAAAAAAAAAAA: ", new_list[1])

LOOOOOOOOOOOOOL:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.
As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.
One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.
The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.
Police have appealed for information about the attack.
Insp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.
"While the exact cause is still under investigation, it is thought that the fire was started deliberately."
BAAAAAAAAAAAAAA:  A blaze alarm clock went off at the Holiday Inn in Hope Street at not far from 04

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


In [ ]:
print(raw_datasets['train'][3])
print(training_set[20003])

{'document': 'John Edward Bates, formerly of Spalding, Lincolnshire, but now living in London, faces a total of 22 charges, including two counts of indecency with a child.\nThe 67-year-old is accused of committing the offences between March 1972 and October 1989.\nMr Bates denies all the charges.\nGrace Hale, prosecuting, told the jury that the allegations of sexual abuse were made by made by four male complainants and related to when Mr Bates was a scout leader in South Lincolnshire and Cambridgeshire.\n"The defendant says nothing of that sort happened between himself and all these individuals. He says they are all fabricating their accounts and telling lies," said Mrs Hale.\nThe prosecutor claimed Mr Bates invited one 15 year old to his home offering him the chance to look at cine films made at scout camps but then showed him pornographic films.\nShe told the jury that the boy was then sexually abused leaving him confused and frightened.\nMrs Hale said: "The complainant\'s recollecti

## Sentence Order

In [ ]:
import spacy
from swaps import Swaps
import numpy as np

swap = Swaps(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = swap.mass_ovs()

KeyboardInterrupt: ignored

In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][1])
print("BAAAAAAAAAAAAAA: ", new_list[1])

LOOOOOOOOOOOOOL:  A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.
As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.
One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.
The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.
Police have appealed for information about the attack.
Insp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.
"While the exact cause is still under investigation, it is thought that the fire was started deliberately."
BAAAAAAAAAAAAAA:  the Holiday Inn went off at A fire alarm in Hope Street at about 04:20 BST on Sat

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


## Inversion

In [ ]:
import spacy
from inversion import Inversion
import numpy as np

inverse = Inversion(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = inverse.mass_inversion()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][2])
print("BAAAAAAAAAAAAAA: ", new_list[2])

LOOOOOOOOOOOOOL:  Ferrari appeared in a position to challenge until the final laps, when the Mercedes stretched their legs to go half a second clear of the red cars.
Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen.
The world champion subsequently escaped punishment for reversing in the pit lane, which could have seen him stripped of pole.
But stewards only handed Hamilton a reprimand, after governing body the FIA said "no clear instruction was given on where he should park".
Belgian Stoffel Vandoorne out-qualified McLaren team-mate Jenson Button on his Formula 1 debut.
Vandoorne was 12th and Button 14th, complaining of a handling imbalance on his final lap but admitting the newcomer "did a good job and I didn't".
Mercedes were wary of Ferrari's pace before qualifying after Vettel and Raikkonen finished one-two in final practice, and their concerns appeared to be well founded as the red cars mixed it with the silver through most of qualifying.
After the first runs, R

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


## Defintion replacement

In [ ]:
import nltk
import spacy
import textacy
import numpy as np
from definition import Definition
import definition

nltk.download('wordnet')
nltk.download('omw-1.4')
rs = textacy.resources.ConceptNet()
rs.download()

definition = Definition(docs=training_set['document'][:2000])
lst = np.array(training_set['document'][:2000])
lst_summary = np.array(training_set['summary'][:2000])
lst_id = np.array(training_set['id'][:2000])
new_list = definition.mass_definition()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
print("LOOOOOOOOOOOOOL: ", training_set['document'][0])
print("BAAAAAAAAAAAAAA: ", new_list[0])

LOOOOOOOOOOOOOL:  The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate t

In [ ]:
print(len(training_set))
print(len(new_list))
for index, item in enumerate(new_list):
  training_set = training_set.add_item({"document": item, "summary": lst_summary[index], "id": lst_id[index]})
print(len(training_set))

20000
2000
22000


In [ ]:
import nltk
import spacy
import textacy
import numpy as np
from definition import Definition
import definition

text = textacy.spacier.core.make_spacy_doc(
    "The fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guest was asked to leave the hotel. As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames. One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland. The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed. Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned. Polouse has appealed for information about the attack.",
    "en_core_web_lg")

nltk.download('wordnet')
nltk.download('omw-1.4')
rs = textacy.resources.ConceptNet()
rs.download()
defo = definition.definition_words(text, ["NOUN", "VERB", "ADV", "ADJ"], 0.75)
definition.make_new_text(defo)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


"The fire fear resulting from the awareness of danger went off at the Holiday Inn in Hope Street at (of quantities) imprecise but fairly close to correct 04:20 BST on Saturday and a visitor to whom hospitality is extended was asked to go away from a place the hotel. As they gathered outside they saw the two buses, parked a place within a region identified relative to a center or reference location-by-a place within a region identified relative to a center or reference location in the a motor vehicle with four wheels; usually propelled by an internal combustion engine a large area of land preserved in its natural state as public property, engulfed by flames. One of the tour groups is from Germany, the other from China and Taiwan. It was their preceding all others in time or space or degree the time after sunset and before sunrise while it is dark outside in Northern Ireland. The the operator of a motor vehicle of one of the buses said a quantifier that can be used with count nouns and i

## visualization texts

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
raw_datasets["train"][0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

## Preprocessing the dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])
raw_datasets["train"] = training_set

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 22000
})

In [ ]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    batch_size=8,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["validation"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=8,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
import numpy as np
import nltk


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=1, callbacks=callbacks
)

   6/2750 [..............................] - ETA: 18:37 - loss: 3.8847WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1952s vs `on_train_batch_end` time: 0.2170s). Check your callbacks.


2750/2750 [==============================] - 1510s 543ms/step - loss: 2.9207 - val_loss: 2.5764 - rouge1: 27.7330 - rouge2: 6.9118 - rougeL: 21.4519 - rougeLsum: 21.4972 - gen_len: 18.8800
